# Example ANN generation and training

From melon doc (https://git.rwth-aachen.de/avt-svt/public/MeLOn/-/blob/master/feedforward%20neural%20network/training/keras/example_training_of_ANN.py) and the ANN paper (https://link.springer.com/content/pdf/10.1007/s10957-018-1396-0.pdf).

## Problem statement:
min $f(x)$

where $f$ is a feed forward neural network. 

The peaks function will be used as a dummy problem and will be approximated by $f$. The peaks function is defined as:

$$ g_{peaks}(x_1, x_2) = 3\cdot(1-x_1)^2 \cdot \exp[-x_1^2-(x_2+1)^2] -10\cdot (x_1/5-x_1^3-x_2^5)\cdot\exp(-x_1^2-x_2^2)- \exp[-(x_1+1)^2-x_2^2]/3$$

$$g_{peaks} : \mathbb{R}^2 \to \mathbb{R}$$

The function dominion is defined as $D = \{x_1, x_2 \in \mathbb{R}: -3 \leq x_1,x_2 \leq 3 \}$

The initial dataset will be generated by Latin hypercube sampling.

In [1]:
import utils
import tensorflow as tf
import numpy as np
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from pyDOE import lhs
import maingopy

Implement peaks function as defined above

In [2]:
def peaks(X):
    term1 = 3*np.multiply((1-X[:,0])**2, np.exp(-(X[:,0])**2-(X[:,1]+1)**2))
    term2 = np.multiply(-10*(X[:,0]/5-X[:,0]**3-X[:,1]**5), np.exp(-X[:,0]**2-X[:,1]**2))
    term3 = -np.exp(-(X[:,0]+1)**2-X[:,1]**2)/3
    y = term1 + term2 + term3
    return np.expand_dims(y, axis=-1)

In [3]:
# test global minimum at f(0.228, -1.626)=-6.551
peaks(np.array([[0.228, -1.626], [0.5,0.5]]))

array([[-6.55112995],
       [ 0.37537558]])

## Training data generation

Generate 500 samples with latin hypercube sampling (package pyDOE), rescale to [-3, 3] and evaluate.

In [4]:
lhd = lhs(2, samples=600)
Xpeaks = lhd*6-3
ypeaks = peaks(Xpeaks)

## Set parameters

In [5]:
problem_name = 'peaks'
# dimensionality of the data
input_dim = 2
output_dim = 1
# scale Input to [-1,1] range
scaleInput = True
# normalize Output to z-score
normalizeOutput = True

define scale and normalize (from melon keras utils)

In [6]:
def scale(X, scaleInput):
    # scale Input values to range [-1,1] in each dimension
    if (scaleInput):
        nom = (X -  X.min(axis=0))*2
        denom = X.max(axis=0) - X.min(axis=0)
        denom[denom==0] = 1
        return -1 + nom/denom
    else:
        return X

def normalize(y, normalizeOutput):
    # normalize output to z-score
    if(normalizeOutput):
        y_norm = (y - np.mean(y, axis=0))/np.std(y, axis=0);
        return y_norm
    else:
        return y

In [7]:
Xnorm = scale(Xpeaks, scaleInput)
ynorm = normalize(ypeaks, normalizeOutput)

In [8]:
# split into test, validation and test sets (70-15-15% split)
Xtrain, Xval, ytrain, yval = train_test_split(Xnorm, ynorm, test_size=0.15)
n_train = Xtrain.shape[0]

## Set output parameters

Two hidden layers of 10, 8 (different from source paper)-> (2x10+1) + (10x8+1) + (8x1)+1 = 127 parameters to optimize.

ReLU activation function for the all activation functions exctept the output layer, which is linear. 

Learning by first-order stochastic gradient descent (ADAM, https://arxiv.org/abs/1412.6980) for 100 epochs with a 0.001 leraning rate (default).

In [9]:
# output filename
output_folder = "./data/Output/"
filename_out = output_folder + problem_name
# training parameters
network_layout = [10, 8]
activation_function = 'relu'
activation_function_out = 'linear'
learning_rate = 0.001
kernel_regularizer = tf.keras.regularizers.l2(l=0.0001)  # L2 regularization penalty
# 'he_normal' for relu activation, 'glorot_uniform' for everything else
kernel_initializer = 'he_normal'
# It draws samples from a truncated normal distribution centered on 0 with stddev = sqrt(2 / fan_in) where fan_in is the number of input units in the weight tensor.
optimizer = 'adam'
epochs = 1000
#batch_size = 128
random_state = 1

## Build model

With the inputs defined above:

keras.sequential: a plain stack of layers where each layer has exactly one input tensor and one output tensor.


In [10]:
# Sequential class: Linear stack of layers.
model = tf.keras.Sequential()
# Create and add first layer
model.add(tf.keras.layers.Dense(network_layout[0],
                                name="input",
                                kernel_initializer=kernel_initializer,
                                kernel_regularizer=kernel_regularizer,
                                activation=activation_function,
                                input_dim=input_dim))
# Create and add all remaining layers (in this case, 9x9 layout)
for neuron in network_layout[1:]:
    model.add(tf.keras.layers.Dense(neuron,
                                    kernel_initializer=kernel_initializer,
                                    kernel_regularizer=kernel_regularizer,
                                    activation=activation_function))
# Output layer w linear function
model.add(tf.keras.layers.Dense(output_dim, name="output",
                                kernel_initializer='glorot_uniform',
                                kernel_regularizer=kernel_regularizer,
                                activation=activation_function_out))

model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae'])
# Generate a table summarizing the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (Dense)               (None, 10)                30        
                                                                 
 dense (Dense)               (None, 8)                 88        
                                                                 
 output (Dense)              (None, 1)                 9         
                                                                 
Total params: 127
Trainable params: 127
Non-trainable params: 0
_________________________________________________________________


## Train ANN model on dataset

First-order gradient-based optimization of stochastic objective functions (https://arxiv.org/abs/1412.6980)

In [ ]:
############################  TRAINING ############################ 

training_time = time.time()

history = model.fit(Xtrain, ytrain, validation_data=(Xval, yval),
                    epochs=epochs, verbose=1)

# history = model.fit(Xtrain, ytrain, validation_data=(Xval, yval),
#                     epochs=epochs, batch_size=batch_size, verbose=1)
training_time = training_time - time.time()

Epoch 1/1000
16/16 [==============================] - 1s 13ms/step - loss: 1.3259 - mse: 1.3212 - mae: 0.8064 - val_loss: 1.0948 - val_mse: 1.0901 - val_mae: 0.7542
Epoch 2/1000
16/16 [==============================] - 0s 2ms/step - loss: 1.1635 - mse: 1.1588 - mae: 0.7339 - val_loss: 0.9385 - val_mse: 0.9338 - val_mae: 0.6865
Epoch 3/1000
16/16 [==============================] - 0s 2ms/step - loss: 1.0768 - mse: 1.0721 - mae: 0.6871 - val_loss: 0.8556 - val_mse: 0.8509 - val_mae: 0.6459
Epoch 4/1000
16/16 [==============================] - 0s 2ms/step - loss: 1.0264 - mse: 1.0218 - mae: 0.6599 - val_loss: 0.8112 - val_mse: 0.8065 - val_mae: 0.6211
Epoch 5/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.9971 - mse: 0.9925 - mae: 0.6451 - val_loss: 0.7780 - val_mse: 0.7734 - val_mae: 0.6089
Epoch 6/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.9736 - mse: 0.9690 - mae: 0.6389 - val_loss: 0.7583 - val_mse: 0.7537 - val_mae: 0.6024
Epoch 7/1000
16

Epoch 51/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.4963 - mse: 0.4907 - mae: 0.4951 - val_loss: 0.4244 - val_mse: 0.4187 - val_mae: 0.4360
Epoch 52/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.4914 - mse: 0.4858 - mae: 0.4925 - val_loss: 0.4262 - val_mse: 0.4205 - val_mae: 0.4395
Epoch 53/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.4884 - mse: 0.4827 - mae: 0.4942 - val_loss: 0.4227 - val_mse: 0.4170 - val_mae: 0.4421
Epoch 54/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.4831 - mse: 0.4774 - mae: 0.4903 - val_loss: 0.4155 - val_mse: 0.4098 - val_mae: 0.4306
Epoch 55/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.4772 - mse: 0.4715 - mae: 0.4862 - val_loss: 0.4210 - val_mse: 0.4152 - val_mae: 0.4401
Epoch 56/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.4740 - mse: 0.4682 - mae: 0.4859 - val_loss: 0.4136 - val_mse: 0.4078 - val_mae: 0.4359
Epoch 57/1

Epoch 101/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.3328 - mse: 0.3261 - mae: 0.4033 - val_loss: 0.3178 - val_mse: 0.3111 - val_mae: 0.3901
Epoch 102/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.3294 - mse: 0.3227 - mae: 0.4000 - val_loss: 0.3144 - val_mse: 0.3077 - val_mae: 0.3861
Epoch 103/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.3282 - mse: 0.3214 - mae: 0.3999 - val_loss: 0.3182 - val_mse: 0.3115 - val_mae: 0.3963
Epoch 104/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.3259 - mse: 0.3191 - mae: 0.3998 - val_loss: 0.3114 - val_mse: 0.3046 - val_mae: 0.3873
Epoch 105/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.3225 - mse: 0.3157 - mae: 0.3979 - val_loss: 0.3091 - val_mse: 0.3023 - val_mae: 0.3884
Epoch 106/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.3209 - mse: 0.3141 - mae: 0.3973 - val_loss: 0.3098 - val_mse: 0.3030 - val_mae: 0.3874
Epoc

16/16 [==============================] - 0s 2ms/step - loss: 0.2363 - mse: 0.2286 - mae: 0.3378 - val_loss: 0.2379 - val_mse: 0.2303 - val_mae: 0.3430
Epoch 151/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2342 - mse: 0.2265 - mae: 0.3368 - val_loss: 0.2371 - val_mse: 0.2294 - val_mae: 0.3461
Epoch 152/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2333 - mse: 0.2256 - mae: 0.3359 - val_loss: 0.2340 - val_mse: 0.2263 - val_mae: 0.3382
Epoch 153/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2322 - mse: 0.2245 - mae: 0.3365 - val_loss: 0.2332 - val_mse: 0.2254 - val_mae: 0.3431
Epoch 154/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2301 - mse: 0.2224 - mae: 0.3338 - val_loss: 0.2332 - val_mse: 0.2254 - val_mae: 0.3402
Epoch 155/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.2307 - mse: 0.2229 - mae: 0.3368 - val_loss: 0.2311 - val_mse: 0.2233 - val_mae: 0.3385
Epoch 156/1000
16/1

16/16 [==============================] - 0s 2ms/step - loss: 0.1694 - mse: 0.1608 - mae: 0.2875 - val_loss: 0.1825 - val_mse: 0.1739 - val_mae: 0.3131
Epoch 200/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.1685 - mse: 0.1599 - mae: 0.2877 - val_loss: 0.1801 - val_mse: 0.1715 - val_mae: 0.3118
Epoch 201/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.1690 - mse: 0.1604 - mae: 0.2894 - val_loss: 0.1790 - val_mse: 0.1704 - val_mae: 0.3124
Epoch 202/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.1671 - mse: 0.1585 - mae: 0.2856 - val_loss: 0.1800 - val_mse: 0.1713 - val_mae: 0.3068
Epoch 203/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.1646 - mse: 0.1559 - mae: 0.2830 - val_loss: 0.1766 - val_mse: 0.1680 - val_mae: 0.3109
Epoch 204/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.1638 - mse: 0.1551 - mae: 0.2830 - val_loss: 0.1770 - val_mse: 0.1683 - val_mae: 0.3105
Epoch 205/1000
16/1

...

16/16 [==============================] - 0s 2ms/step - loss: 0.0698 - mse: 0.0592 - mae: 0.1890 - val_loss: 0.0965 - val_mse: 0.0859 - val_mae: 0.2240
Epoch 396/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0698 - mse: 0.0592 - mae: 0.1882 - val_loss: 0.0984 - val_mse: 0.0878 - val_mae: 0.2330
Epoch 397/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0700 - mse: 0.0594 - mae: 0.1903 - val_loss: 0.0969 - val_mse: 0.0862 - val_mae: 0.2224
Epoch 398/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0689 - mse: 0.0583 - mae: 0.1884 - val_loss: 0.0951 - val_mse: 0.0845 - val_mae: 0.2217
Epoch 399/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0689 - mse: 0.0582 - mae: 0.1883 - val_loss: 0.0968 - val_mse: 0.0862 - val_mae: 0.2214
Epoch 400/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0681 - mse: 0.0575 - mae: 0.1875 - val_loss: 0.0960 - val_mse: 0.0854 - val_mae: 0.2275
Epoch 401/1000
16/1

16/16 [==============================] - 0s 2ms/step - loss: 0.0623 - mse: 0.0514 - mae: 0.1769 - val_loss: 0.0848 - val_mse: 0.0739 - val_mae: 0.2056
Epoch 445/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0619 - mse: 0.0511 - mae: 0.1750 - val_loss: 0.0847 - val_mse: 0.0738 - val_mae: 0.2098
Epoch 446/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0627 - mse: 0.0518 - mae: 0.1783 - val_loss: 0.0831 - val_mse: 0.0723 - val_mae: 0.2040
Epoch 447/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0630 - mse: 0.0522 - mae: 0.1797 - val_loss: 0.0881 - val_mse: 0.0772 - val_mae: 0.2072
Epoch 448/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0640 - mse: 0.0531 - mae: 0.1778 - val_loss: 0.0857 - val_mse: 0.0748 - val_mae: 0.2109
Epoch 449/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0628 - mse: 0.0519 - mae: 0.1765 - val_loss: 0.0837 - val_mse: 0.0728 - val_mae: 0.2042
Epoch 450/1000
16/1

16/16 [==============================] - 0s 2ms/step - loss: 0.0555 - mse: 0.0444 - mae: 0.1617 - val_loss: 0.0743 - val_mse: 0.0632 - val_mae: 0.1866
Epoch 494/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0558 - mse: 0.0447 - mae: 0.1618 - val_loss: 0.0741 - val_mse: 0.0630 - val_mae: 0.1900
Epoch 495/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0559 - mse: 0.0448 - mae: 0.1639 - val_loss: 0.0725 - val_mse: 0.0614 - val_mae: 0.1878
Epoch 496/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0555 - mse: 0.0444 - mae: 0.1613 - val_loss: 0.0727 - val_mse: 0.0616 - val_mae: 0.1884
Epoch 497/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0551 - mse: 0.0440 - mae: 0.1617 - val_loss: 0.0733 - val_mse: 0.0622 - val_mae: 0.1830
Epoch 498/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0552 - mse: 0.0440 - mae: 0.1604 - val_loss: 0.0738 - val_mse: 0.0627 - val_mae: 0.1873
Epoch 499/1000
16/1

16/16 [==============================] - 0s 2ms/step - loss: 0.0505 - mse: 0.0393 - mae: 0.1511 - val_loss: 0.0655 - val_mse: 0.0542 - val_mae: 0.1711
Epoch 543/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0503 - mse: 0.0391 - mae: 0.1496 - val_loss: 0.0700 - val_mse: 0.0588 - val_mae: 0.1826
Epoch 544/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0510 - mse: 0.0397 - mae: 0.1499 - val_loss: 0.0646 - val_mse: 0.0533 - val_mae: 0.1708
Epoch 545/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0507 - mse: 0.0394 - mae: 0.1512 - val_loss: 0.0656 - val_mse: 0.0544 - val_mae: 0.1736
Epoch 546/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0497 - mse: 0.0385 - mae: 0.1484 - val_loss: 0.0664 - val_mse: 0.0552 - val_mae: 0.1753
Epoch 547/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0499 - mse: 0.0387 - mae: 0.1484 - val_loss: 0.0642 - val_mse: 0.0529 - val_mae: 0.1688
Epoch 548/1000
16/1

16/16 [==============================] - 0s 2ms/step - loss: 0.0456 - mse: 0.0342 - mae: 0.1381 - val_loss: 0.0598 - val_mse: 0.0484 - val_mae: 0.1603
Epoch 592/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0457 - mse: 0.0343 - mae: 0.1362 - val_loss: 0.0599 - val_mse: 0.0485 - val_mae: 0.1610
Epoch 593/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0456 - mse: 0.0342 - mae: 0.1386 - val_loss: 0.0628 - val_mse: 0.0514 - val_mae: 0.1696
Epoch 594/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0463 - mse: 0.0349 - mae: 0.1388 - val_loss: 0.0611 - val_mse: 0.0497 - val_mae: 0.1685
Epoch 595/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0460 - mse: 0.0346 - mae: 0.1403 - val_loss: 0.0607 - val_mse: 0.0493 - val_mae: 0.1624
Epoch 596/1000
16/16 [==============================] - 0s 2ms/step - loss: 0.0458 - mse: 0.0343 - mae: 0.1370 - val_loss: 0.0598 - val_mse: 0.0484 - val_mae: 0.1621
Epoch 597/1000
16/1

In [ ]:
# Save entire model to a HDF5 file
model.save(filename_out + '_model.h5')
# Save model to XML
utils.save_model_to_xml(filename_out + '.xml', model, Xpeaks, ypeaks, scaleInput, normalizeOutput)
# plot predictions
y_pred = model.predict(Xnorm)

## Plot surrogate function

In [ ]:
############################ PLOT PREDICTIONS ############################ 
n_train = Xnorm.shape[0]
n_rt = np.sqrt(n_train).astype(int)
X = np.zeros((n_rt, n_rt))
Y = np.zeros((n_rt, n_rt))
Z_true = np.zeros((n_rt, n_rt))
Z_pred = np.zeros((n_rt, n_rt))
for i in range(n_rt):
    for j in range(n_rt):
        X[i, j] = Xnorm[i * n_rt + j, 0]
        Y[i, j] = Xnorm[i * n_rt + j, 1]
        Z_true[i, j] = ynorm[i * n_rt + j]
        Z_pred[i, j] = y_pred[i * n_rt + j]

fig = plt.figure(figsize=plt.figaspect(0.4))
ax = fig.add_subplot(1, 2, 1, projection='3d')
cmap = plt.get_cmap('coolwarm')
ax.plot_surface(X, Y, Z_true, cmap=cmap)
ax.set_title('training data')

ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.plot_surface(X, Y, Z_pred)
ax.set_title('learned function')
plt.show()


## Optimize surrogate model with maingopy

Melon computes the relaxations of the feed forward network ````maingopy.melonpy.FeedForwardNet()```` needed by the B&B solver by providing envelopes of the activation functions (currently supported: purelin, linear, tanh, tansig, relu, relu6). The procedure is the same with gaussian processes and support vector machines.

We load the trained model information from the generated XML file (see utils for details). which will be the optimization problem parameters. Then, we define the problem bounds of $D$ (see problem definition) in maingopy. Lastly, we define the evaluation function for maingopy.

Maingopy then takes the envelopes of the ANN and performs a global deterministic optimization (it is  B&B solver with a reduced space formulation for these envelopes, speeding up computation: https://www.avt.rwth-aachen.de/global/show_document.asp?id=aaaaaaaaabclahw). 

In [ ]:
# To define a model, we need to spcecialize the MAiNGOmodel class
class Model(maingopy.MAiNGOmodel):
    def __init__(self):
        maingopy.MAiNGOmodel.__init__(self)
        # Initialize feedforward neural network and load data from example csv file
        self.ffANN = maingopy.melonpy.FeedForwardNet()
        # folder where the model xml is stored
        self.path = "./data/Output/"
        # xml filename
        self.fileName= "peaks"
        # open them (define that it is an XML instead of a CSV)
        self.ffANN.load_model(self.path, self.fileName, maingopy.melonpy.XML)

    # We need to implement the get_variables functions for specifying the optimization varibles
    def get_variables(self):
        # define bounds of the original variables, so that it rescales the results fo the optimization
        # the optimization is done with the normalized version of these values
        variables = [ maingopy.OptimizationVariable(maingopy.Bounds(-3,3), maingopy.VT_CONTINUOUS, "x"),
                      maingopy.OptimizationVariable(maingopy.Bounds(-3,3), maingopy.VT_CONTINUOUS, "y") ]
        return variables

    # We need to implement the evaluate function that computes the values of the objective and constraints from the variables.
    # Note that the variables in the 'vars' argument of this function do correspond to the optimization variables defined in the get_variables function.
    # However, they are different objects for technical reasons. The only mapping we have between them is the position in the list.
    # The results of the evaluation (i.e., objective and constraint values) need to be return in an EvaluationContainer
    def evaluate(self, vars):
        x = vars[0]
        y = vars[1]
        
        # Inputs to the ANN are the variables x and y
        annInputs = [x, y]
        
        # Evaluate the network (in reduced-space)
        # This returns a list, because the output of the network may be multidimensional
        annOutputs = self.ffANN.calculate_prediction_reduced_space(annInputs)

        # Set the ANN output (only 1 in this case) as objective to be minimized
        result = maingopy.EvaluationContainer()
        result.objective = annOutputs[0]

        return result

In [ ]:
# To work with the problem, we first create an instance of the model.
myModel = Model()
# We then create an instance of MAiNGO, the solver, and hand it the model.
myMAiNGO = maingopy.MAiNGO(myModel)

In [ ]:
# We can have MAiNGO read a settings file:
#fileName = ""
#myMAiNGO.read_settings(fileName) # If fileName is empty, MAiNGO will attempt to open MAiNGOSettings.txt
myMAiNGO.set_log_file_name(".logs/my_log_file.log")
myMAiNGO.set_option("writeCsv", True)
myMAiNGO.set_iterations_csv_file_name(".logs/iterations.csv")
myMAiNGO.set_solution_and_statistics_csv_file_name(".logs/solution_and_statistics.csv")

You can see the input in form of a GAMS file in case you are interested in ````./logs/my_problem_file_MAiNGO.gms````.

In [ ]:
myMAiNGO.write_model_to_file_in_other_language(writingLanguage=maingopy.LANG_GAMS, fileName="./logs/my_problem_file_MAiNGO.gms", solverName="SCIP", writeRelaxationOnly=False)

### Solution

In [ ]:
# Finally, we call the solve routine to solve the problem.
maingoStatus = myMAiNGO.solve()
print(maingoStatus)

In [ ]:
print("Global optimum: f([{}, {}]) = {}".format(myMAiNGO.get_solution_point()[0], myMAiNGO.get_solution_point()[1], myMAiNGO.get_objective_value()))

In [ ]:
print("ANN value at true minimum = g(0.228, -1.626) = {}".format(myMAiNGO.evaluate_model_at_point(np.array([0.228, -1.626]))))